In [2]:
from scipy.sparse import hstack
import pandas as pd
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from transformers import BertTokenizer, BertModel
import torch

drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/DATASET/movies.csv')

data['overview'] = data['overview'].str.lower()
data['overview'] = data['overview'].str.replace('[^\w\s]','')

data['keywords'] = data['keywords'].str.lower()
data['keywords'] = data['keywords'].str.replace('[^\w\s]','')

tfidf_overview = TfidfVectorizer(stop_words='english')
tfidf_matrix_overview = tfidf_overview.fit_transform(data['overview'].fillna(''))

tfidf_keywords = TfidfVectorizer(stop_words='english')
tfidf_matrix_keywords = tfidf_keywords.fit_transform(data['keywords'].fillna(''))

tfidf_combined = hstack((tfidf_matrix_overview, tfidf_matrix_keywords))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def extract_features(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    features = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return features

movie_overviews = data['overview'].fillna('').tolist()
movie_keywords = data['keywords'].fillna('').tolist()
movie_texts = [overview + ' ' + keywords for overview, keywords in zip(movie_overviews, movie_keywords)]
movie_features = [extract_features(text) for text in movie_texts]

movie_features = torch.tensor(movie_features)
cosine_sim = linear_kernel(movie_features, movie_features)

def recommend_similar_movies(movie_title, num_recommendations=5):
    movie_index = data[data['original_title'] == movie_title].index[0]
    similar_movies = list(enumerate(cosine_sim[movie_index]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]
    similar_movie_indices = [i[0] for i in similar_movies]
    return data['original_title'].iloc[similar_movie_indices]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-1b195a2b3d31>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  data['overview'] = data['overview'].str.replace('[^\w\s]','')
<ipython-input-2-1b195a2b3d31>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  data['keywords'] = data['keywords'].str.replace('[^\w\s]','')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<ipython-input-2-1b195a2b3d31>:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  movie_features = torch.tensor(movie_features)


In [34]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/DATASET/movies.csv')

def find_similar_movie(movie_title):
    for title in data['original_title']:
        if movie_title.lower() in title.lower():
            return title
    return None

selected_movie_title = input("Enter the title of the selected movie: ")
found_movie = find_similar_movie(selected_movie_title)

if found_movie:
    recommendations = recommend_similar_movies(found_movie)
    print(f"\nRecommendations for the movie '{found_movie}':\n")
    for movie in recommendations:
        print(f"Movie: {movie}")
        movie_index = data[data['original_title'] == movie].index[0]
        print(f"Overview: {data['overview'].iloc[movie_index]}")
        print("—" * 350)
else:
    print(f"Movie '{selected_movie_title}' not found. Please try entering another title.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter the title of the selected movie: Batman

Recommendations for the movie 'Batman v Superman: Dawn of Justice':

Movie: The Dark Knight
Overview: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————